In [14]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
import calendar
from global_land_mask import globe
import glob
import math
from scipy import stats
import os
import netCDF4 as nc

from highres_read import read_var_hires
from myReadGCMsDaily import read_var_mod
from regrid_wght_3d import regrid_wght_wnans

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
plt.clf()
plt.rcParams['figure.figsize'] = (10, 10)

<Figure size 1000x1000 with 0 Axes>

In [2]:
from con_models import get_cons
con, use_colors, varname, pvarname, modname, warming_modname, hiresmd, amip_md = get_cons()

In [3]:
#latitude range
latr1 = 30
latr2 = 80


In [ ]:
from obs_data_function import obs
merwind, macwind, temp, sfctemp, sfcpres, p_lev_obs, p_mer_lat, merlon, merlev = obs('surface_winter','T2M', 700, latr1, latr2)
theta_700 = np.array(np.multiply(temp, (100000/(merlev[p_lev_obs]*100))**(con)))
theta_sfc = np.array(np.multiply(sfctemp, (100000/sfcpres)**(con)))
p_CAOI    = np.array(np.subtract(theta_sfc,theta_700))

#Mask for the ocean
maskm = np.ones((len(p_mer_lat),len(merlon)))

for a in range(len(p_mer_lat)):
    for b in range(len(merlon)):
        if globe.is_land(p_mer_lat[a], merlon[b])==True:
            maskm[a,b] = math.nan
    ##############################
    
plot_CAOI = np.array(np.multiply(maskm,p_CAOI))

num_UM = []
num_M  = []

M_plot = np.array(plot_CAOI)

for j in range(np.shape(M_plot)[1]):
    count_UM = 0

    for i in range(np.shape(M_plot)[0]):
        for k in range(np.shape(M_plot)[2]):
            if M_plot[i,j,k]>=-9:
                count_UM = count_UM+1
    num_UM.append(count_UM)

for j in range(np.shape(M_plot)[1]):
    count_M = 0

    for i in range(np.shape(M_plot)[0]):
        for k in range(np.shape(M_plot)[2]):
            if M_plot[i,j,k]!=np.nan:
                count_M = count_M+1
    num_M.append(count_M)

num_M = np.array(num_M)
num_UM = np.array(num_UM)

f = np.divide(num_UM,num_M)


In [ ]:
plt.clf()
mpl.rcParams['figure.dpi'] = 100
plt.gcf().set_size_inches(6.4, 4.8)
plt.plot(p_mer_lat,f,color='blue',linestyle='dashed')
plt.scatter(p_mer_lat,f,color='blue',label='Observations:MERRA2',marker='x')
plt.xlabel('latitude')
plt.title('CAO zonal mean ditribution: Winter')
plt.legend()
plt.savefig('../figures/final/winter_M_lats.png')

In [ ]:
plt.legend()
plt.xlabel('latitude')
plt.title('seasonal CAO zonal mean ditribution/for observations: MERRA2')
plt.savefig('../figures/final/new_all_M_lats.png')

In [ ]:
plt.clf()
mpl.rcParams['figure.dpi'] = 100
plt.gcf().set_size_inches(6.4, 4.8)
plt.title('seasonal CAO zonal mean ditribution\nfor observations: MERRA2')
plt.savefig('../figures/final/test.png')

### GCMs

In [12]:
time1=[2010, 1, 1]
time2=[2010, 2, 28]

In [5]:
test_num = 0
for j in range(test_num,test_num+1):
    print(modname[j],' ', str(j))
    for i in varname:
        locals()[i+'__'+str(j+1)] = read_var_mod('surface', modname[j], 'historical', i, time1, time2)

    for k in pvarname:
        locals()[k+'__'+str(j+1)] = read_var_mod('p_level', modname[j], 'historical', k, time1, time2)

print('done')

CESM2   0


/glade/work/geethma/research/codes/myReadGCMsDaily.py:73: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  data.extend(np.array(datai[ind1:ind2+1,:,:]))


done


In [6]:
# mm = len(modname)
# for j in range(0,len(hiresmd)):
#     print(hiresmd[j],' ', str(j))
#     for i in varname:
#         locals()[i+'__'+str(j+1+mm)] = read_var_hires('surface', hiresmd[j], 'highresSST-present', i, time1, time2)

#     for k in pvarname:
#         locals()[k+'__'+str(j+1+mm)] = read_var_hires('p_level', hiresmd[j], 'highresSST-present', k, time1, time2)

# print('done')

In [6]:
M_plot = []
   

In [7]:
for i in range(test_num,test_num+1): #l,mm+len(hiresmd)
    if i<len(modname):
        print(modname[i],str(i))
    else:
        print(hiresmd[i-mm],str(i))

    for j in varname:
        lat  = locals()[j+'__'+str(i+1)][0]
        lon  = locals()[j+'__'+str(i+1)][1]
        time = locals()[j+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        #print(j)
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        locals()['plot_'+j+str(i+1)] = np.array(np.multiply(maskm,locals()[j+str(i+1)][:,lat_ind1[0]:lat_ind2[0],:]))
        locals()['grid_'+j+str(i+1)] = locals()['plot_'+j+str(i+1)]
        #regrid_wght_wnans(lats,lon,locals()['plot_'+j+str(i+1)],lats_edges,lons_edges)[0]

    for k in pvarname:
        #print(k)
        lat  = locals()[k+'__'+str(i+1)][0]
        lon  = locals()[k+'__'+str(i+1)][1]
        time = locals()[k+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        locals()['plot_levels'+str(i+1)] = locals()['ta__'+str(i+1)][3]
        locals()['grid_'+k+str(i+1)] = []

        levels = locals()['plot_levels'+str(i+1)]

        for p in range(len(levels)):
            if levels[p] == 70000:
                #print(levels[p])
                locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
                locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
                temp_700   = np.array(np.multiply(maskm,locals()[k+str(i+1)][:,p,lat_ind1[0]:lat_ind2[0],:]))
                # regrid     = regrid_wght_wnans(lats,lon,temp_700,lats_edges,lons_edges)
                # grid_t_700 = regrid[0]
                # lat_n      = regrid[2][:,0]
                break;

    theta_700 = temp_700*(100000/70000)**con
    # theta_700 = grid_t_700*(100000/70000)**con
    theta_t2m = locals()['grid_tas'+str(i+1)]*(100000/locals()['grid_psl'+str(i+1)])**con

    t = min(len(theta_t2m),len(theta_700))
    M_700  = theta_t2m[0:t,:,:] - theta_700[0:t,:,:]
    M_plot.extend(M_700)

   

CESM2 0


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [ ]:
num_UM = []
num_M  = []

In [ ]:
M_plot = np.array(M_plot)

for j in range(np.shape(M_plot)[1]):
    count_UM = 0
    
    for i in range(np.shape(M_plot)[0]):
        for k in range(np.shape(M_plot)[2]):
            if M_plot[i,j,k]>=-9:
                count_UM = count_UM+1
    num_UM.append(count_UM)

In [ ]:
for j in range(np.shape(M_plot)[1]):
    count_M = 0
    
    for i in range(np.shape(M_plot)[0]):
        for k in range(np.shape(M_plot)[2]):
            if M_plot[i,j,k]!=np.nan:
                count_M = count_M+1
    num_M.append(count_M)


In [ ]:
num_M = np.array(num_M)
num_UM = np.array(num_UM)

In [ ]:
f = np.divide(num_UM,num_M)

In [ ]:
plt.plot(lats,f,color=use_colors[test_num],label=modname[test_num],linestyle='solid')

In [ ]:
# plt.plot(lats,f,color=use_colors[test_num+3],label='autumn',linestyle='dashed')

In [ ]:
plt.legend()
plt.xlabel('latitude')
plt.title('seasonal CAO zonal mean ditribution\n for cesm2')
# plt.savefig('../figures/final/cesm2_M_lats.png')

In [ ]:
DJF = []
DJF_lats = []

In [1]:
#Append all models DJF data to DJF

In [ ]:
count = 1000
for i in range(len(DJF_lats_warm)):
    temp  = len(DJF_lats_warm[i])
    count = min(count,temp)
        

In [ ]:
warm_mean = np.nanmean(DJF_warm,axis=0)

In [ ]:
np.shape(DJF_warm[0])

In [ ]:
count = 30

In [ ]:
from scipy import stats
warm_new = []
warm_lats_new = []

for i in range(len(DJF_lats_warm)):
    bin_means, bin_edges, binnumber       = stats.binned_statistic(DJF_lats_warm[i], DJF_warm[i],      'mean', bins=count, range=(29.,80.5))
    bin_means_c, bin_edges_c, binnumber_c = stats.binned_statistic(DJF_lats_warm[i], DJF_lats_warm[i], 'mean', bins=count, range=(29.,80.5))
    warm_new.append(bin_means)
    warm_lats_new.append(bin_means_c)
    

In [ ]:
from scipy import stats
hist_new = []
hist_lats_new = []

for i in range(len(DJF_lats)):
    bin_means, bin_edges, binnumber       = stats.binned_statistic(DJF_lats[i], DJF[i],      'mean', bins=count, range=(29.,80.5))
    bin_means_c, bin_edges_c, binnumber_c = stats.binned_statistic(DJF_lats[i], DJF_lats[i], 'mean', bins=count, range=(29.,80.5))
    hist_new.append(bin_means)
    hist_lats_new.append(bin_means_c)
    

In [ ]:
hist_mean = np.nanmean(hist_new,axis=0)
hist_lats_mean = np.nanmean(hist_lats_new,axis=0)

In [ ]:
warm_mean = np.nanmean(warm_new,axis=0)
warm_lats_mean = np.nanmean(warm_lats_new,axis=0)

In [ ]:
plt.plot(warm_lats_mean, warm_mean, linestyle='dashed', color='red', label='warm-mean line',linewidth=2)
plt.plot(hist_lats_mean, hist_mean, linestyle='dashed', color='darkblue', label='historical-mean line',linewidth=2)
plt.legend()
plt.xlabel('latitude')
plt.title('seasonal CAO zonal mean ditribution\nhistorical and warming')
plt.savefig('../figures/final/warming_M_lats.png')

In [ ]:
red_colors = [ 'rosybrown', 'lightcoral', 'indianred', 'brown', 'firebrick', 'maroon', 'darkred', 'red', 'mistyrose', 'salmon', 'tomato', 'darksalmon', 
              'coral', 'orangered', 'lightsalmon', 'sienna', 'seashell', 'chocolate', 'saddlebrown', 'sandybrown', 'peachpuff', 'peru', 'linen', 'bisque', 
              'darkorange', 'burlywood', 'antiquewhite', 'tan', 'navajowhite', 'blanchedalmond']

In [ ]:
blue_colors = [ 'lavender', 'midnightblue', 'navy', 'darkblue', 'mediumblue', 'blue', 'slateblue', 
               'darkslateblue', 'mediumslateblue', 'mediumpurple', 'rebeccapurple', 'blueviolet', 'indigo', 
               'darkorchid', 'darkviolet', 'mediumorchid', 'thistle', 'plum', 'violet', 'purple']

In [ ]:
for i in range(len(DJF_lats)):
    plt.plot(warm_lats_new[i], warm_new[i], color=red_colors[i], alpha=0.2)
    plt.plot(hist_lats_new[i], hist_new[i], color=blue_colors[i], alpha=0.2)
plt.plot(warm_lats_mean, warm_mean, linestyle='dashed', color='red', label='warm-mean line',linewidth=2)
plt.plot(hist_lats_mean, hist_mean, linestyle='dashed', color='darkblue', label='historical-mean line',linewidth=2)
plt.legend()
plt.xlabel('latitude')
plt.title('seasonal CAO zonal mean ditribution\nhistorical and warming')
plt.savefig('../figures/final/warming_M_lats.png')   

In [ ]:
np.savez('/glade/work/geethma/research/npzfilesn/CAO_frequency.npz', DJF=DJF, DJF_lats=DJF_lats, DJF_lats_mean=DJF_lats_mean, 
                    DJF_mean=DJF_mean)

#u10_monthdateyear.npz

In [ ]:
fn = glob.glob('/glade/work/geethma/research/npzfilesn/CAO_frequency.npz')

In [ ]:
d = np.load(fn[0])
DJF = np.array(d['DJF'],allow_pickle=True)


In [ ]:
import csv

# open the file in the write mode
f = open('/glade/work/geethma/research/codes/csv_file', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(DJF_lats_mean)

# close the file
f.close()

In [ ]:
import csv

header = ['DJF','DJF_lats']

with open('csv/hist_DJF.csv', 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(header)

    # write a row to the csv file
    writer.writerow(DJF)
    writer.writerow(DJF_lats)

In [ ]:
import pandas as pd
url = '/glade/work/geethma/research/codes/csv_file'
data = pd.read_csv(url)

In [23]:
fn = '/glade/scratch/geethma/cmip6/surface/CMIP6.CMIP.NCAR.CESM2-WACCM-FV2.abrupt-4xCO2.r1i1p1f1.day.tas.gn.v20200403.tas_day_CESM2-WACCM-FV2_abrupt-4xCO2_r1i1p1f1_gn_01210101-01310110.nc'

In [24]:
data = xr.open_dataset(fn)

/glade/u/home/geethma/.local/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [19]:
time = data.time

In [20]:
time

<xarray.DataArray 'time' (time: 3661)>
array([cftime.DatetimeNoLeap(140, 12, 22, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(140, 12, 23, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(140, 12, 24, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(150, 12, 30, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(150, 12, 31, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(151, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 0140-12-22 00:00:00 ... 0151-01-01 00:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    standard_name:  time
    title:          time
    type:           double